In [1]:
import osmnx as ox
import geopandas as gpd
import matplotlib.pyplot as plt
from folium import Map, CircleMarker, LayerControl

# Step 1: Define the region (Victoria, Australia)
place_name = "Victoria, Australia"

# Step 2: Download amenities (hospitals) and shops (supermarkets)
# Hospital-like amenities
hospitals = ox.geometries_from_place(place_name, tags={"amenity": ["hospital", "clinic", "doctors"]})

# Supermarkets and related shops
# supermarkets = ox.geometries_from_place(place_name, tags={"shop": ["supermarket", "grocery"]})

#leisure

#entertainment

#beach

#parks






/tmp/ipykernel_373361/2867471040.py:11: FutureWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in the v2.0.0 release. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  hospitals = ox.geometries_from_place(place_name, tags={"amenity": ["hospital", "clinic", "doctors"]})
/home/robertus/.local/lib/python3.10/site-packages/osmnx/_overpass.py:254: UserWarning: This area is 121 times your configured Overpass max query area size. It will automatically be divided up into multiple sub-queries accordingly. This may take a long time.
  multi_poly_proj = utils_geo._consolidate_subdivide_geometry(poly_proj)


In [2]:
supermarkets = ox.geometries_from_place(place_name, tags={"shop": ["supermarket", "grocery"]})

/tmp/ipykernel_373361/3449639386.py:1: FutureWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in the v2.0.0 release. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  supermarkets = ox.geometries_from_place(place_name, tags={"shop": ["supermarket", "grocery"]})
/home/robertus/.local/lib/python3.10/site-packages/osmnx/_overpass.py:254: UserWarning: This area is 121 times your configured Overpass max query area size. It will automatically be divided up into multiple sub-queries accordingly. This may take a long time.
  multi_poly_proj = utils_geo._consolidate_subdivide_geometry(poly_proj)


Make sure to filter out Nan as that is the name of some locations

In [4]:
# Step 3: Subsample 1000 of each
hospitals_sample = hospitals.sample(1000)
supermarkets_sample = supermarkets.sample(1000)

# Step 4: Convert to GeoDataFrame if needed (sometimes the data is already a GeoDataFrame)
hospitals_gdf = gpd.GeoDataFrame(hospitals_sample, geometry="geometry")
supermarkets_gdf = gpd.GeoDataFrame(supermarkets_sample, geometry="geometry")

In [5]:
import folium

# Step 1: Create a base map centered around Victoria, Australia
victoria_center = [-37.4713, 144.7852]  # Approximate center of Victoria
m = folium.Map(location=victoria_center, zoom_start=8)

# Convert polygon geometries to points (centroids)
hospitals_gdf['geometry'] = hospitals_gdf['geometry'].centroid
supermarkets_gdf['geometry'] = supermarkets_gdf['geometry'].centroid

# Step 2: Add hospitals (in red) and supermarkets (in blue) to the map
for idx, row in hospitals_gdf.iterrows():
    folium.CircleMarker(
        location=[row.geometry.y, row.geometry.x],
        radius=5,
        color='red',
        fill=True,
        fill_color='red',
        popup=f"Hospital: {row['name']}" if 'name' in row else "Hospital"
    ).add_to(m)

for idx, row in supermarkets_gdf.iterrows():
    folium.CircleMarker(
        location=[row.geometry.y, row.geometry.x],
        radius=5,
        color='blue',
        fill=True,
        fill_color='blue',
        popup=f"Supermarket: {row['name']}" if 'name' in row else "Supermarket"
    ).add_to(m)

# Step 3: Add a layer control
folium.LayerControl().add_to(m)

# Step 4: Display the map
m.save("hospitals_supermarkets_map.html")
m


/tmp/ipykernel_373361/3285583513.py:8: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  hospitals_gdf['geometry'] = hospitals_gdf['geometry'].centroid
/tmp/ipykernel_373361/3285583513.py:9: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  supermarkets_gdf['geometry'] = supermarkets_gdf['geometry'].centroid


Still not fixed but in case you need other variables for each location

In [2]:
amenities_gdf = ox.geometries_from_place(
    place_name,
    tags={'amenity': amenities, 'addr:street': True, 'addr:postcode': True}
)

# Extract relevant details: amenity type, name, street address, postcode, and geolocation
amenities_gdf = amenities_gdf[['amenity', 'name', 'addr:street', 'addr:postcode', 'geometry']]

# Add latitude and longitude columns
amenities_gdf['latitude'] = amenities_gdf.geometry.y
amenities_gdf['longitude'] = amenities_gdf.geometry.x

# Save the DataFrame to CSV
amenities_gdf.to_csv("victoria_amenities_with_address.csv")


/tmp/ipykernel_87626/963248693.py:1: FutureWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in the v2.0.0 release. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  amenities_gdf = ox.geometries_from_place(
/home/robertus/.local/lib/python3.10/site-packages/osmnx/_overpass.py:254: UserWarning: This area is 121 times your configured Overpass max query area size. It will automatically be divided up into multiple sub-queries accordingly. This may take a long time.
  multi_poly_proj = utils_geo._consolidate_subdivide_geometry(poly_proj)


KeyboardInterrupt: 